- This program merges OEC (with opportunity variables) and GTA dataset

# TODO

- Investigate left-only merge
  - parece que os hs4_id do df_gta estão tirando leading 0 e daí o produto id fica errado e não merge
  - descobrir onde está o problema dos hs4_id do GTA. Talvez tenha que voltar no código que trata os dados

- Investigate right-only merge
  - descobrir porque a base OEC não tem 400 produtos da base GTA

# Loading packages

In [1]:
import numpy as np
import pandas as pd
import os
from google.colab import drive

# Setting the ambience

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
work_dir                            = '/content/gdrive/My Drive/ip_complexity/create_finaldataset'
output_dir                          = '/content/gdrive/My Drive/ip_complexity/create_finaldataset/output'
create_dataset_gta_dir              = '/content/gdrive/My Drive/ip_complexity/create_dataset_gta/output/data'
create_variables_opportunity_dir    = '/content/gdrive/My Drive/ip_complexity/oec_data/create_variables_opportunity/output/data'

In [4]:
pd.set_option('display.float_format', '{:.4f}'.format) # only shows 2 decimal numbers

# Oppening OEC dataset

In [5]:
df_oec = pd.read_csv(create_variables_opportunity_dir + "/dataset_oec_with_opportunity.csv")

In [6]:
# Convert 'hs4_id' column to string and fill with zeros until it reachs 4 digits
df_oec["hs4_id"] = df_oec["hs4_id"].astype(str).str.zfill(4)

In [7]:
df_oec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3882912 entries, 0 to 3882911
Data columns (total 13 columns):
 #   Column               Dtype  
---  ------               -----  
 0   year                 int64  
 1   country_id           object 
 2   country              object 
 3   country_eci          float64
 4   hs4_id               object 
 5   hs4_name             object 
 6   hs4_pci              float64
 7   tv_relatedness       float64
 8   tv_rca               float64
 9   opportunity_index_1  float64
 10  opportunity_index_2  float64
 11  opportunity_index_3  float64
 12  tv_rca_bigger_1      int64  
dtypes: float64(7), int64(2), object(4)
memory usage: 385.1+ MB


In [8]:
df_oec = df_oec.sort_values(by = 'hs4_id')

In [9]:
df_oec[df_oec["hs4_id"] == '6602']

,year,country_id,country,country_eci,hs4_id,hs4_name,hs4_pci,tv_relatedness,tv_rca,opportunity_index_1,opportunity_index_2,opportunity_index_3,tv_rca_bigger_1


# Oppening GTA dataset

In [10]:
df_gta = pd.read_csv(create_dataset_gta_dir + "/dataset_gta_intervention_product.csv", sep = ";", dtype= str)

## Cleaning

In [11]:
# Define the renaming function
rename_func = lambda x: x.lower().replace(' ', '_')
# Rename the columns using the function
df_gta = df_gta.rename(columns=rename_func)

In [12]:
df_gta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6574565 entries, 0 to 6574564
Data columns (total 11 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   state_act_id               object
 1   intervention_id            object
 2   state_act_title            object
 3   announcement_date          object
 4   gta_evaluation             object
 5   currently_in_force         object
 6   implementing_jurisdiction  object
 7   intervention_type          object
 8   mast_chapter               object
 9   affected_sectors           object
 10  affected_product           object
dtypes: object(11)
memory usage: 551.8+ MB


In [13]:
df_gta = df_gta.rename(columns = {"implementing_jurisdiction": "country",
                         "affected_product": "hs4_id"
                        })

In [14]:
# Extracting numbers before "."
df_gta['hs4_id'] = df_gta['hs4_id'].str.split('.').str[0]


In [15]:
# Convert "announcement_data" column to date type
df_gta["announcement_date"] = pd.to_datetime(df_gta["announcement_date"])

# Create a variable "year" with the year from "announcement_data"
df_gta["year"] = df_gta["announcement_date"].dt.year

In [16]:
# Extract the first four digits from 'hs4_id' to merge with OEC data
df_gta["hs4_id"] = df_gta["hs4_id"].astype(str).str[:4]

In [17]:
# drop duplicates
df_gta = df_gta.drop_duplicates()

## Normalizing countries' names

In [18]:
# Extract unique country names from both datasets
oec_countries = set(df_oec['country'])
gta_countries = set(df_gta['country'])

# Find the countries with different names
different_countries = oec_countries.symmetric_difference(gta_countries)

# Sort the different countries in alphabetical order
sorted_countries = sorted(different_countries)

In [19]:
# Create a mapping dictionary for country renaming
country_mapping = {
    'United States of America': 'United States',
    'Republic of Korea': 'South Korea',
    'Republic of the Sudan': 'Sudan',
    'Congo': 'Republic of the Congo',
    #'': 'North Korea',
    'DR Congo': 'Democratic Republic of the Congo',
    'Republic of Moldova': 'Moldova',
    'Lao': 'Laos',
    'Ivory Coast': "Cote d'Ivoire'",
    'Myanmar': 'Burma',
    'Bosnia & Herzegovina': 'Bosnia and Herzegovina' 
    }

    # Add more country mappings as needed

df_gta['country'] = df_gta['country'].apply(lambda x: country_mapping.get(x, x))

In [20]:
# Drop observations that I could not normalize between both datasets
df_gta = df_gta[~(df_gta['country'].isin(gta_countries) & df_gta['country'].isin(different_countries))]

In [21]:
# Drop observations that affect no product
df_gta = df_gta[df_gta["hs4_id"] != '0nan']
df_gta = df_gta[df_gta["hs4_id"] != '000n']
df_gta = df_gta[df_gta["hs4_id"] != 'nan']

In [22]:
df_gta.hs4_id.unique()

array(['2710', '6302', '6303', ..., '8134', '8135', '8140'], dtype=object)

In [23]:
len(np.sort(df_gta.hs4_id.unique()))

1450

In [24]:
len(df_oec.hs4_id.unique())

1062

In [25]:
# Extract unique country names from both datasets
oec_countries = set(df_oec.hs4_id.unique())
gta_countries = set(df_gta.hs4_id.unique())

# Find the countries with different names
different_countries = oec_countries.symmetric_difference(gta_countries)

# Sort the different countries in alphabetical order
sorted_countries = sorted(different_countries)
for product in sorted_countries:
  if product in oec_countries:
    print(product)

1519
2851
4109
8485
8520
8524
9009
9501
9502


In [26]:
for product in sorted_countries:
  if product in gta_countries:
    print(product)

0308
0501
0502
0506
0507
0508
0510
0812
0814
0903
0907
1002
1012
1013
1019
1022
1023
1029
1031
1039
1041
1042
1051
1059
1061
1062
1063
1064
1069
1203
1213
1401
1503
1505
1521
1522
1603
1802
1903
2011
2012
2013
2021
2022
2023
2031
2032
2041
2042
2043
2044
2045
2050
2061
2062
2063
2064
2068
2069
2071
2072
2074
2075
2076
2081
2083
2084
2085
2086
2089
2091
2099
2109
2305
2307
2502
2509
2512
2513
2514
2525
2528
2611
2617
2621
2705
2802
2806
2808
2813
2816
2820
2824
2831
2848
2852
2853
2911
2913
3011
3019
3021
3022
3023
3024
3025
3027
3028
3029
3031
3032
3033
3034
3035
3036
3038
3039
3043
3044
3045
3046
3047
3048
3049
3051
3052
3053
3054
3055
3056
3057
3061
3062
3071
3072
3073
3074
3075
3076
3077
3078
3079
3081
3082
3083
3089
3201
3205
3211
3601
3605
3704
3803
3805
3807
3813
3824
3825
3826
4003
4004
4017
4021
4022
4029
4031
4039
4041
4049
4051
4052
4059
4061
4062
4063
4064
4069
4071
4072
4079
4081
4089
4090
4100
4112
4113
4114
4115
4206
4304
4404
4405
4406
4413
4417
4501
4502
4812
4822
4904


# Merging 

In [27]:
df = pd.merge(df_gta, df_oec, on=['hs4_id', 'year', 'country'], how = "outer", indicator = True)

In [28]:
df = df[df["year"] < 2022]

In [29]:
df["_merge"].value_counts() # right_only means that a product p was not protected or liberalized by country c at year j. It is ok to drop, since we're interested only in the products that are target by policies


right_only    3581674
both          2831527
left_only       86508
Name: _merge, dtype: int64

- Left_only
  - Maybe left_only this is happening because Country c enacted an act that harms product p, but Country c do not exports product p. I'll try to find an example.

In [30]:
df[(df["_merge"] == "left_only")].year.value_counts()

2020    13314
2019    11692
2015    10520
2013    10054
2014     9554
2012     5513
2011     5329
2018     4490
2010     3489
2017     3291
2016     3200
2021     3051
2009     2023
2008      988
Name: year, dtype: int64

In [31]:
df[(df["_merge"] == "left_only") & (df["country"] == "Brazil")].year.value_counts()

2015    728
2009    262
2014    225
2011    199
2010    149
2013    148
2021    127
2020     69
2017     38
2019     19
2018     12
2012      7
2016      2
Name: year, dtype: int64

In [32]:
df['gta_evaluation'].unique()

array(['Red', 'Green', 'Amber', nan], dtype=object)

In [33]:
df[(df["_merge"] == "left_only") & (df['gta_evaluation'] == "Red") ].country.value_counts().head(50)

China             16106
India              5539
Germany            2775
France             2665
United States      2198
Italy              2076
United Kingdom     1922
Spain              1702
Netherlands        1398
Belgium            1288
Brazil             1149
Turkey             1068
Poland             1035
Switzerland        1010
Austria             957
Russia              944
Czechia             910
Sweden              901
Denmark             862
Romania             848
Portugal            835
Indonesia           834
Ireland             830
Greece              813
Slovakia            809
Hungary             808
Finland             780
Bulgaria            773
Slovenia            759
Lithuania           751
Croatia             730
Argentina           726
Japan               571
South Korea         233
Pakistan            154
Singapore           150
Norway              145
Kazakhstan          138
Thailand            123
Vietnam             121
Mexico              111
Australia       

- This list is very interesting! If my interpretation is right, these countries are using harmfull protection against products that they don't even export!

In [34]:
df[(df["_merge"] == "left_only")].hs4_id.value_counts().head(50)

8487    13019
3824    10972
8486     3014
2806     2197
6006     1794
9619     1616
6004     1276
3826     1183
4113     1071
6005      911
1521      852
4112      840
1603      799
8110      731
5204      710
4114      682
1505      603
0308      572
0812      560
6814      556
2820      531
0506      531
3201      519
2853      513
1002      511
3805      500
9206      482
7319      467
7316      466
5004      460
5310      459
5505      458
5909      458
1522      451
5306      438
5511      437
5605      435
7806      431
0814      429
1213      426
7904      423
8435      422
5506      421
2831      409
6213      407
6003      407
2813      406
4417      406
0510      404
2816      398
Name: hs4_id, dtype: int64

In [35]:
df[(df["_merge"] == "left_only") & (df['gta_evaluation'] == "Red")].T

,3982,3983,3984,3985,3986,3987,4782,4783,4941,4942,...,3191070,3191071,3191072,3191073,3191074,3191075,3216955,3217634,3217635,3217636
state_act_id,123,123,123,123,123,123,157,157,157,157,...,71927,71927,71927,71927,71927,71927,73154,73221,73221,73221
intervention_id,14539,14539,14539,14539,14539,14539,15366,15366,15366,15366,...,115180,115180,115180,115180,115180,115180,117177,117277,117277,117277
state_act_title,Ukraine: Import duty surcharges up to 13% to r...,Ukraine: Import duty surcharges up to 13% to r...,Ukraine: Import duty surcharges up to 13% to r...,Ukraine: Import duty surcharges up to 13% to r...,Ukraine: Import duty surcharges up to 13% to r...,Ukraine: Import duty surcharges up to 13% to r...,India: Incentives for leather and textile sect...,India: Incentives for leather and textile sect...,India: Incentives for leather and textile sect...,India: Incentives for leather and textile sect...,...,Japan: Government launches the Program for Pro...,Japan: Government launches the Program for Pro...,Japan: Government launches the Program for Pro...,Japan: Government launches the Program for Pro...,Japan: Government launches the Program for Pro...,Japan: Government launches the Program for Pro...,Germany: Funding guideline for the support of ...,Canada: Government launches a CAD 1.5 billion ...,Canada: Government launches a CAD 1.5 billion ...,Canada: Government launches a CAD 1.5 billion ...
announcement_date,2009-03-04 00:00:00,2009-03-04 00:00:00,2009-03-04 00:00:00,2009-03-04 00:00:00,2009-03-04 00:00:00,2009-03-04 00:00:00,2009-04-01 00:00:00,2009-04-01 00:00:00,2009-04-01 00:00:00,2009-04-01 00:00:00,...,2020-05-22 00:00:00,2020-05-22 00:00:00,2020-05-22 00:00:00,2020-05-22 00:00:00,2020-05-22 00:00:00,2020-05-22 00:00:00,2021-10-04 00:00:00,2021-06-21 00:00:00,2021-06-21 00:00:00,2021-06-21 00:00:00
gta_evaluation,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red,...,Red,Red,Red,Red,Red,Red,Red,Red,Red,Red
currently_in_force,no,no,no,no,no,no,no,no,no,no,...,no,no,no,no,no,no,yes,yes,yes,yes
country,Ukraine,Ukraine,Ukraine,Ukraine,Ukraine,Ukraine,India,India,India,India,...,Japan,Japan,Japan,Japan,Japan,Japan,Germany,Canada,Canada,Canada
intervention_type,Import tariff,Import tariff,Import tariff,Import tariff,Import tariff,Import tariff,Export subsidy,Export subsidy,Export subsidy,Export subsidy,...,Financial grant,Financial grant,Financial grant,Financial grant,Financial grant,Financial grant,Financial assistance in foreign market,State loan,State loan,State loan
mast_chapter,Tariff measures,Tariff measures,Tariff measures,Tariff measures,Tariff measures,Tariff measures,P6 Export-support measures,P6 Export-support measures,P6 Export-support measures,P6 Export-support measures,...,L Subsidies (excluding export subsidies under P6),L Subsidies (excluding export subsidies under P6),L Subsidies (excluding export subsidies under P6),L Subsidies (excluding export subsidies under P6),L Subsidies (excluding export subsidies under P6),L Subsidies (excluding export subsidies under P6),"P9 Export measures, n.e.s.",L Subsidies (excluding export subsidies under P6),L Subsidies (excluding export subsidies under P6),L Subsidies (excluding export subsidies under P6)
affected_sectors,"013, 211, 241, 242, 281, 282, 293, 296, 392, 4...","049, 211, 215, 232, 241, 271, 272, 281, 379, 3...","211, 215, 271, 281, 282, 296, 341, 439, 448, 4...","013, 049, 211, 215, 271, 281, 282, 293, 294, 3...","211, 281, 379, 432, 439, 448, 491","281, 282, 379, 432, 448, 491","019, 261, 263, 264, 265, 266, 267, 268, 271, 2...","019, 261, 263, 264, 265, 266, 267, 268, 271, 2...","019, 261, 263, 264, 266, 267, 271, 272, 279, 2...","019, 261, 263, 264, 265, 266, 267, 268, 271, 2...",...,"336, 342, 431, 432, 439, 449, 452, 464, 471, 4...","342, 362, 431, 432, 439, 449, 452, 464, 471, 4...","336, 431, 432, 439, 448, 449, 452, 461, 464, 4...","336, 342, 421, 431, 432, 439, 448, 449, 452, 4...","342, 362, 421, 431, 432, 439, 448, 449, 452, 4...","449, 472",342,"333, 35

In [36]:
#df = df[df["_merge"] != "right_only"]

In [37]:
#df = df.drop(columns = '_merge')

In [38]:
# Drop the DataFrame from memory
#del df_gta
#del df_oec

In [39]:
# create protectionist/liberal dummies
# Create a dummy variable based on the condition 'color == "Red"'
df['protectionist'] = df['gta_evaluation'].apply(lambda x: 1 if x == 'Red' else (0 if x == 'Green' else np.nan))
df['liberal'] = df['gta_evaluation'].apply(lambda x: 1 if x == 'Green' else (0 if x == 'Red' else np.nan))
df['amber'] = df['gta_evaluation'].apply(lambda x: 1 if x == 'Green' else (0 if x == 'Amber' else np.nan))

In [40]:
df.describe()

,year,country_eci,hs4_pci,tv_relatedness,tv_rca,opportunity_index_1,opportunity_index_2,opportunity_index_3,tv_rca_bigger_1,protectionist,liberal
count,6499709.0000,5533819.0000,6035265.0000,6413201.0000,6413201.0000,6035265.0000,6035265.0000,6035265.0000,6413201.0000,2826695.0000,2826695.0000
mean,2011.2014,0.4180,0.0940,0.2301,1.3263,0.0254,-0.6171,-0.0935,0.2456,0.7312,0.2688
std,7.4689,0.9362,1.0143,0.1591,19.7114,0.2938,21.4713,3.4307,0.4305,0.4433,0.4433
min,1995.0000,-2.9401,-3.5684,0.0000,0.0000,-1.8689,-9527.6967,-1533.9267,0.0000,0.0000,0.0000
25%,2006.0000,-0.2181,-0.6826,0.0916,0.0138,-0.1040,-0.0724,-0.0136,0.0000,0.0000,0.0000
50%,2013.0000,0.5614,0.2276,0.2002,0.2417,0.0158,0.0001,0.0000,0.0000,1.0000,0.0000
75%,2018.0000,1.0708,0.8913,0.3662,0.9782,0.1607,0.2526,0.0556,0.0000,1.0000,1.0000
max,2021.0000,2.2607,3.1534,0.9479,14382.4219,2.8062,4423.4044,696.9246,1.0000,1.0000,1.0000


# Saving

In [41]:
# Specify the desired column order
column_order = ['year', 'country', 'country_id', 'country_eci', 'state_act_id', 'intervention_id', 'protectionist', 'liberal']  # Add more variables as needed

# Reorder the DataFrame columns
df = df[column_order + list(df.columns.difference(column_order))]

In [42]:
df.to_csv(output_dir + '/data/oec_gta_dataset.csv', index = False)